## Tensorflow Hub 사용해서 text classification

In [1]:
import numpy as np
import tensorflow as tf
!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

     |████████████████████████████████| 4.0 MB 8.5 MB/s 


In [2]:
print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

버전:  2.5.0
즉시 실행 모드:  True
허브 버전:  0.12.0
GPU 사용 가능


In [4]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:80%]', 'train[80%:]', 'test'),
    as_supervised=True)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


#### train을 전체 데이터의 80%, validation은 20%를 할당한다.

In [8]:
samp_data, samp_label = next(iter(train_data.batch(10)))

In [9]:
samp_data

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [14]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [], 
                           dtype = tf.string, trainable = True)
hub_layer(samp_data[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

#### 텐서플로 허브에는 여러개의 **사전 훈련된 텍스트 임베딩 모델**이 있다.

In [15]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                672       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 400,725
Trainable params: 400,725
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [17]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs = 30, validation_data = validation_data.batch(512))

Epoch 1/30
40/40 [==============================] - 6s 56ms/step - loss: 4.7396 - acc: 0.5646 - val_loss: 3.9383 - val_acc: 0.6004
Epoch 2/30
40/40 [==============================] - 2s 41ms/step - loss: 3.5554 - acc: 0.6174 - val_loss: 3.0632 - val_acc: 0.6392
Epoch 3/30
40/40 [==============================] - 2s 44ms/step - loss: 4.6313 - acc: 0.5989 - val_loss: 5.8721 - val_acc: 0.5534
Epoch 4/30
40/40 [==============================] - 2s 45ms/step - loss: 4.7993 - acc: 0.5915 - val_loss: 5.1835 - val_acc: 0.5662
Epoch 5/30
40/40 [==============================] - 2s 45ms/step - loss: 4.1241 - acc: 0.6069 - val_loss: 2.8886 - val_acc: 0.6704
Epoch 6/30
40/40 [==============================] - 2s 44ms/step - loss: 2.7754 - acc: 0.6787 - val_loss: 2.6408 - val_acc: 0.6800
Epoch 7/30
40/40 [==============================] - 2s 44ms/step - loss: 5.6730 - acc: 0.5613 - val_loss: 7.3319 - val_acc: 0.5088
Epoch 8/30
40/40 [==============================] - 2s 45ms/step - loss: 7.3388 - a

In [18]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 1.0095 - acc: 0.7597
loss: 1.009
acc: 0.760
